In [1]:
# The code was removed by Watson Studio for sharing.

# Coursera Capstone Project

### This notebook will be used for my final project at the end of the IBM Data Science Professional Certificate

#### Import the required libraries

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    widgetsnbextension-3.5.1   |   py37h89c1867_4         1.8 MB  conda-f

In [34]:
import pandas as pd
path = 'https://github.com/NickyRichards123/Coursera_Capstone/blob/643bbde7aaf8277749ec12f70a39424fbcebc3ac/doogal_london_postcodes.csv?raw=true '
dfpostcodes = pd.read_csv(path)

Import list of postcodes
make list into dataframe for postcode and neighbourhood name

In [35]:
dfpostcodes.head()

,Unnamed: 0,Southern districts
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
1,SE2,Abbey Wood
2,SE3,"Blackheath, Westcombe Park"
3,SE4,"Brockley, Crofton Park, Honor Oak Park"
4,SE5,Camberwell


In [36]:
dfpostcodes=dfpostcodes.rename(columns={"Unnamed: 0" : "PostcodeArea","Southern districts":"Neighbourhoods"})
dfpostcodes.head()

,PostcodeArea,Neighbourhoods
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
1,SE2,Abbey Wood
2,SE3,"Blackheath, Westcombe Park"
3,SE4,"Brockley, Crofton Park, Honor Oak Park"
4,SE5,Camberwell


In [37]:
dfpostcodes.sort_values('PostcodeArea', 
                      inplace=True, 
                      ascending=True)

Import csv file of postcode lat/lon
clean the file, get rid of surplus columns and rows
take average of postcodes to find centre of each neighbourhood
stitch lat lon with postcodes dataframe

In [38]:
dfpostcodes.head()

,PostcodeArea,Neighbourhoods
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
9,SE10,Greenwich
10,SE11,Lambeth
11,SE12,"Grove Park, Lee"
12,SE13,"Hither Green, Lewisham"


In [39]:
dfpostcodes.reset_index(drop=True, inplace=True)
dfpostcodes.head()

,PostcodeArea,Neighbourhoods
0,SE1,"Bermondsey, Borough, Southwark, Waterloo"
1,SE10,Greenwich
2,SE11,Lambeth
3,SE12,"Grove Park, Lee"
4,SE13,"Hither Green, Lewisham"


In [74]:
path2= 'https://github.com/NickyRichards123/Coursera_Capstone/blob/155a3b8f8c73268ed338e2e1f81d551ba0b8e3f2/london_postcodes-ons-postcodes-directory.csv?raw=true'
dflatlon = pd.read_csv(path2)

In [75]:
dflatlon.head()

,pcd,reg,lat,long
0,SE1 0AA,SE1,51.498708,-0.101772
1,SE1 0AB,SE1,51.504038,-0.095542
2,SE1 0AD,SE1,51.499568,-0.101016
3,SE1 0AE,SE1,51.498575,-0.104112
4,SE1 0AF,SE1,51.498206,-0.104098


In [76]:
dflatlon=dflatlon.drop('pcd', axis=1)


In [77]:
dflatlon=dflatlon.rename(columns={"reg" : "PostcodeArea","lat":"latitude", "long":"longitude"})
dflatlon.head()

,PostcodeArea,latitude,longitude
0,SE1,51.498708,-0.101772
1,SE1,51.504038,-0.095542
2,SE1,51.499568,-0.101016
3,SE1,51.498575,-0.104112
4,SE1,51.498206,-0.104098


In [78]:
dflatlon.dtypes

PostcodeArea     object
latitude        float64
longitude       float64
dtype: object

In [79]:
dfgroup=dflatlon.groupby("PostcodeArea",as_index=False).mean()

In [67]:
dfgroup

,PostcodeArea1,latitude,longitude
0,SE1,51.498191,-0.090293
1,SE10,51.484002,0.000746
2,SE11,51.489510,-0.109062
3,SE12,51.446044,0.025921
4,SE13,51.459298,-0.010488
5,SE14,51.475470,-0.043112
6,SE15,51.472734,-0.067061
7,SE16,51.496560,-0.054826
8,SE17,51.488515,-0.094691
9,SE18,51.485410,0.071126


In [53]:
dfgroup.dtypes

PostcodeArea1     object
latitude         float64
longitude        float64
dtype: object

In [82]:
Londondata=pd.merge(dfgroup, dfpostcodes, how='inner', left_on='PostcodeArea', right_on='PostcodeArea',left_index=True)

In [83]:
Londondata

,PostcodeArea,latitude,longitude,Neighbourhoods
0,SE1,51.498191,-0.090293,"Bermondsey, Borough, Southwark, Waterloo"
1,SE10,51.484002,0.000746,Greenwich
2,SE11,51.489510,-0.109062,Lambeth
3,SE12,51.446044,0.025921,"Grove Park, Lee"
4,SE13,51.459298,-0.010488,"Hither Green, Lewisham"
5,SE14,51.475470,-0.043112,"New Cross, New Cross Gate"
6,SE15,51.472734,-0.067061,"Nunhead, Peckham"
7,SE16,51.496560,-0.054826,"Rotherhithe, South Bermondsey, Surrey Docks"
8,SE17,51.488515,-0.094691,"Elephant & Castle, Walworth"
9,SE18,51.485410,0.071126,"Plumstead, Woolwich"


,PostcodeArea,Neighbourhoods,PostcodeArea1,latitude,longitude
0,SE1,"Bermondsey, Borough, Southwark, Waterloo",SE1,51.498191,-0.090293
1,SE10,Greenwich,SE10,51.484002,0.000746
2,SE11,Lambeth,SE11,51.489510,-0.109062
3,SE12,"Grove Park, Lee",SE12,51.446044,0.025921
4,SE13,"Hither Green, Lewisham",SE13,51.459298,-0.010488


,PostcodeArea,Neighbourhoods,PostcodeArea1,latitude,longitude
0,SE1,"Bermondsey, Borough, Southwark, Waterloo",SE1,51.498191,-0.090293
1,SE10,Greenwich,SE10,51.484002,0.000746
2,SE11,Lambeth,SE11,51.489510,-0.109062
3,SE12,"Grove Park, Lee",SE12,51.446044,0.025921
4,SE13,"Hither Green, Lewisham",SE13,51.459298,-0.010488


In [61]:
address = 'South Norwood, UK'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of South Norwood are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of South Norwood are 51.3980951, -0.0768216.


In [62]:
# create map of TOronto using latitude and longitude values
map_SLondon = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, pcd, neighborhood in zip(Londondata['latitude'], Londondata['longitude'], Londondata['PostcodeArea'], Londondata['Neighbourhoods']):
    label = '{}, {}'.format(neighborhood, pcd)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_SLondon)  
    
map_SLondon

In [63]:
CLIENT_ID = 'NFJ23WYXHZIJYO0AWKR3MVAO1NCCAMFSFQ2GZE3UWY1W0I3D' # your Foursquare ID
CLIENT_SECRET = 'SRZMYEIGS2YNJKL5O1PKFX44E2ZQIISRR2ZEORU3ON4AWZ1J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NFJ23WYXHZIJYO0AWKR3MVAO1NCCAMFSFQ2GZE3UWY1W0I3D
CLIENT_SECRET:SRZMYEIGS2YNJKL5O1PKFX44E2ZQIISRR2ZEORU3ON4AWZ1J


In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhoods', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
SLondonvenues=getNearbyVenues(names=Londondata['Neighbourhoods'], 
                                 latitudes=Londondata['latitude'], 
                                 longitudes=Londondata['longitude'])

Bermondsey, Borough, Southwark, Waterloo
Greenwich
Lambeth
Grove Park, Lee
Hither Green, Lewisham
New Cross, New Cross Gate
Nunhead, Peckham
Rotherhithe, South Bermondsey, Surrey Docks
Elephant & Castle, Walworth
Plumstead, Woolwich
Crystal Palace, Upper Norwood
Abbey Wood
Anerley, Penge
Dulwich
East Dulwich
Forest Hill
Herne Hill
South Norwood
Sydenham
Tulse Hill, West Norwood
Thamesmead
Blackheath, Westcombe Park
Brockley, Crofton Park, Honor Oak Park
Camberwell
Bellingham, Catford, Hither Green
Charlton
Deptford
Eltham, Mottingham
West Brompton, World's End
Battersea, Clapham Junction
Balham
Barnes, Castelnau
East Sheen, Mortlake
Putney, Roehampton
Norbury, Streatham
Tooting
Earlsfield, Wandsworth
Merton, Wimbledon
Brixton, Streatham Hill
Raynes Park, South Wimbledon
Brompton, Chelsea
Clapham
Earl's Court
Fulham, Parson's Green
South Kensington
Nine Elms, South Lambeth
Brixton, Stockwell


In [85]:
print(SLondonvenues.shape)
SLondonvenues.head()

(3058, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bermondsey, Borough, Southwark, Waterloo",51.498191,-0.090293,Tabard Gardens,51.498925,-0.089586,Park
1,"Bermondsey, Borough, Southwark, Waterloo",51.498191,-0.090293,The Roebuck,51.498109,-0.090621,Pub
2,"Bermondsey, Borough, Southwark, Waterloo",51.498191,-0.090293,Spit and Sawdust,51.494850,-0.088640,Pub
3,"Bermondsey, Borough, Southwark, Waterloo",51.498191,-0.090293,Empire Square,51.500332,-0.091031,Residential Building (Apartment / Condo)
4,"Bermondsey, Borough, Southwark, Waterloo",51.498191,-0.090293,Fine Foods,51.498385,-0.084026,Deli / Bodega


In [86]:
# one hot encoding
SLon_onehot = pd.get_dummies(SLondonvenues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SLon_onehot['Neighbourhoods'] = SLondonvenues['Neighbourhoods'] 

# move neighborhood column to the first column
fixed_columns = [SLon_onehot.columns[-1]] + list(SLon_onehot.columns[:-1])
SLon_onehot = SLon_onehot[fixed_columns]

SLon_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Newsagent,Nightclub,Noodle House,Observatory,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Rugby Pitch,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Transportation Service,Tree,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,"Bermondsey, Borough, Southwark, Waterloo",0,0,

In [87]:
print(SLon_onehot.shape)

(3058, 275)


In [90]:
SLon_grouped = SLon_onehot.groupby('Neighbourhoods').mean().reset_index()
SLon_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hill,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,New American Restaurant,Newsagent,Nightclub,Noodle House,Observatory,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Rugby Pitch,Sake Bar,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Transportation Service,Tree,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Abbey Wood,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhoods'] = SLon_grouped['Neighbourhoods']

for ind in np.arange(SLon_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SLon_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Abbey Wood,Supermarket,Historic Site,Campground,Grocery Store,Train Station,Platform,Playground,Accessories Store,Soccer Field,Film Studio,Fried Chicken Joint,French Restaurant,Diner,Forest,Food Truck
1,"Anerley, Penge",Grocery Store,Pub,Pizza Place,Convenience Store,Train Station,Supermarket,Fast Food Restaurant,Park,Coffee Shop,Café,Hardware Store,Garden Center,Tapas Restaurant,Gas Station,Farm
2,Balham,Coffee Shop,Pub,Grocery Store,Bakery,Pizza Place,Indian Restaurant,Supermarket,Italian Restaurant,Café,Sandwich Place,Bar,Bus Stop,Breakfast Spot,Restaurant,Asian Restaurant
3,"Barnes, Castelnau",Pub,Park,Coffee Shop,Farmers Market,Café,Restaurant,Grocery Store,Platform,Pizza Place,Gastropub,Athletics & Sports,Food & Drink Shop,Community Center,Train Station,Thai Restaurant
4,"Battersea, Clapham Junction",Pub,Café,Bakery,Italian Restaurant,Coffee Shop,Cocktail Bar,Bar,Portuguese Restaurant,Pizza Place,Grocery Store,Supermarket,Indian Restaurant,Gym / Fitness Center,French Restaurant,Thai Restaurant


## Run K-Means clustering to cluster the neighbourhoods into 6 clusters

In [94]:
# set number of clusters
kclusters = 6

SLon_grouped_clustering = SLon_grouped.drop('Neighbourhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SLon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([4, 3, 1, 2, 5, 3, 1, 2, 5, 1, 1, 5, 1, 3, 5], dtype=int32)

Create another dataframe which shows the cluster details as well as the most common venues


In [96]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SLon_merged = Londondata

SLon_merged = SLon_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhoods')

SLon_merged.head() 

ValueError: cannot insert Cluster Labels, already exists

Explore bedminster
explore postcodes
Group bedminster and postcodes by similarity  (KNN)

In [22]:
address = 'Bedminster, UK'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bedminster are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bedminster are 51.4367393, -2.6042197.
